# Policy Responses to the Coronavirus Pandemic

> [https://github.com/BMClab/covid19](https://github.com/BMClab/covid19)  
> [Laboratory of Biomechanics and Motor Control](http://pesquisa.ufabc.edu.br/bmclab/)  
> Federal University of ABC, Brazil

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Environment" data-toc-modified-id="Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Environment</a></span></li></ul></li><li><span><a href="#Load-dataset-of-running-activities" data-toc-modified-id="Load-dataset-of-running-activities-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load dataset of running activities</a></span><ul class="toc-item"><li><span><a href="#Subject's-information" data-toc-modified-id="Subject's-information-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Subject's information</a></span></li></ul></li><li><span><a href="#Location" data-toc-modified-id="Location-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Location</a></span></li><li><span><a href="#Policy-responses-indexes" data-toc-modified-id="Policy-responses-indexes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Policy responses indexes</a></span><ul class="toc-item"><li><span><a href="#Load-dataset-for-the-different-indexes" data-toc-modified-id="Load-dataset-for-the-different-indexes-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Load dataset for the different indexes</a></span></li></ul></li><li><span><a href="#The-policy-responses-indexes-are-correlated" data-toc-modified-id="The-policy-responses-indexes-are-correlated-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>The policy responses indexes are correlated</a></span></li><li><span><a href="#Weight-policy-responses-indexes-by-number-of-athletes'-countries" data-toc-modified-id="Weight-policy-responses-indexes-by-number-of-athletes'-countries-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Weight policy responses indexes by number of athletes' countries</a></span><ul class="toc-item"><li><span><a href="#Athletes'-countries-that-are-not-on-the-Covid-19-stringency-index-list" data-toc-modified-id="Athletes'-countries-that-are-not-on-the-Covid-19-stringency-index-list-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Athletes' countries that are not on the Covid-19 stringency index list</a></span></li><li><span><a href="#Delete-countries-in-indexes-list-that-are-not-from-athletes" data-toc-modified-id="Delete-countries-in-indexes-list-that-are-not-from-athletes-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Delete countries in indexes list that are not from athletes</a></span><ul class="toc-item"><li><span><a href="#Covid-19-stringency-index-averaged-by-percentage-of-athletes'-countries" data-toc-modified-id="Covid-19-stringency-index-averaged-by-percentage-of-athletes'-countries-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Covid-19 stringency index averaged by percentage of athletes' countries</a></span></li><li><span><a href="#Export-data" data-toc-modified-id="Export-data-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Export data</a></span></li><li><span><a href="#data-normalization" data-toc-modified-id="data-normalization-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>data normalization</a></span></li><li><span><a href="#Group-data-by-week-or-seven-days" data-toc-modified-id="Group-data-by-week-or-seven-days-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Group data by week or seven days</a></span></li></ul></li></ul></li></ul></div>

## Setup

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
%load_ext watermark  

%watermark
%watermark --iversions

### Environment

In [ ]:
path2 = r'./../data/'

pd.set_option('display.float_format', lambda x: '%.4g' % x)
plt.rcParams.update({'font.size': 14, 'xtick.labelsize': 12, 'ytick.labelsize': 12})
sns.set_style('whitegrid', rc={'xtick.bottom': True, 'xtick.top': True, 'ytick.left': True,
              'ytick.right': True, 'xtick.direction': 'in', 'ytick.direction': 'in'})
colors = sns.color_palette()
colors

## Load dataset of running activities

In [ ]:
df = pd.read_parquet(os.path.join(path2, 'run_ww_2020_d.parquet'))
df['athlete'] = df['athlete'].astype('category')  # bug in parquet

### Subject's information

In [ ]:
df = df[['athlete', 'gender', 'age_group', 'country', 'major']
       ].drop_duplicates(subset='athlete').sort_values('athlete').reset_index(drop=True)
df

In [ ]:
df.info(memory_usage='deep')

## Location

In [ ]:
print('Number of countries: {}'.format(df['country'].unique().size))
print('Number of athletes with unknown country: {} ({:.1f} %)'.format(df['country'].isnull().sum(),
       df['country'].isnull().sum()/df['country'].size*100))

In [ ]:
cn_pct = 100*df['country'].value_counts(normalize=True)
cn_pct = cn_pct.to_frame().reset_index()
cn_pct = cn_pct.rename(columns={'index':'country', 'country':'percentage'})
cn_pct['country'] = cn_pct['country'].astype('object')
cn_pct.head(10)

In [ ]:
y = cn_pct.copy(deep=True)
y.loc[y['percentage'] < 1, 'country'] = 'Other countries'
fig = px.pie(y, values='percentage', names='country', labels='percentage',
             title='Countries of the athletes')
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()

## Policy responses indexes

Data from [Policy Responses to the Coronavirus Pandemic](https://ourworldindata.org/policy-responses-covid).

- si: [Government Stringency Index](https://ourworldindata.org/covid-government-stringency-index#government-stringency-index)  
- ci: [Containment and Health Index](https://ourworldindata.org/covid-government-stringency-index#containment-and-health-index)  
- hi: [Stay-at-Home Restrictions](https://ourworldindata.org/covid-stay-home-restrictions)  
- fi: [Workplace Closures](https://ourworldindata.org/covid-school-workplace-closures#workplace-closures)

### Load dataset for the different indexes

In [ ]:
si = pd.read_csv(os.path.join(path2, 'covid-stringency-index.csv'), sep=',',
                 header=0, parse_dates=['Date'], verbose=False, 
                 usecols=['Entity', 'Date', 'stringency_index'])
ci = pd.read_csv(os.path.join(path2, 'covid-containment-and-health-index.csv'), sep=',',
                 header=0, parse_dates=['Date'], verbose=False, 
                 usecols=['Entity', 'Date', 'containment_index'])
hi = pd.read_csv(os.path.join(path2, 'stay-at-home-covid.csv'), sep=',',
                 header=0, parse_dates=['Date'], verbose=False, 
                 usecols=['Entity', 'Date', 'stay_home_requirements'])
wi = pd.read_csv(os.path.join(path2, 'workplace-closures-covid.csv'), sep=',',
                 header=0, parse_dates=['Date'], verbose=False, 
                 usecols=['Entity', 'Date', 'workplace_closures'])
c19idx = si.set_index(['Entity', 'Date']).join([ci.set_index(['Entity', 'Date']),
                                                hi.set_index(['Entity', 'Date']),
                                                wi.set_index(['Entity', 'Date'])])
c19idx.reset_index(inplace=True)
c19idx.rename(columns={'Entity': 'country', 'Date': 'date', 'stringency_index':'si',
                       'containment_index': 'ci', 'stay_home_requirements': 'hi',
                       'workplace_closures': 'wi'},
              inplace=True)
c19idx = c19idx[c19idx['date'].dt.year == 2020]
c19idx['country'].replace({"Cote d'Ivoire": 'Ivory Coast', 'Timor': 'East Timor'},
                          inplace=True)
c19idx

In [ ]:
c19idxm = c19idx.groupby('date').mean()
c19idxm = (c19idxm - c19idxm.min()) / (c19idxm.max() - c19idxm.min())
c19idxm.plot(lw=3, figsize=(12, 4))
plt.show()

## The policy responses indexes are correlated 

In [ ]:
corr = c19idx.groupby('date').mean().corr()
ax = sns.heatmap(corr, annot=True, cbar=False)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=18, rotation=0)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=18, rotation=0)
plt.show()

## Weight policy responses indexes by number of athletes' countries

### Athletes' countries that are not on the Covid-19 stringency index list

In [ ]:
y = cn_pct[~cn_pct['country'].isin(c19idx['country'].unique().tolist())]
print('Athletes not in stringency list: {:.2f} %'.format(y['percentage'].sum()))
display(y)

### Delete countries in indexes list that are not from athletes

In [ ]:
c19idx = c19idx[c19idx['country'].isin(cn_pct['country'])]
c19idx

#### Covid-19 stringency index averaged by percentage of athletes' countries

In [ ]:
fun = lambda row: row[['si', 'ci', 'hi', 'wi']]*cn_pct[cn_pct['country']==row.iloc[0]['country']
                                                      ]['percentage'].values[0]
factor = c19idx['country'].value_counts().size / 100
c19idx2 = c19idx.groupby('country').apply(fun)*factor
c19idx2.rename(columns={'si': 'si_w', 'ci': 'ci_w', 'hi':'hi_w', 'wi':'wi_w'}, inplace=True)
c19idx_w = pd.concat((c19idx[['country', 'date']], c19idx2), axis=1)
c19idx_w

In [ ]:
c19idxm_w = c19idx_w.groupby('date').mean()
c19idxm_w.plot(lw=3, figsize=(12, 4))
plt.show()

#### Export data

In [ ]:
c19idxm_w.to_csv(os.path.join(path2, 'policy_response_indexes.csv'))

#### data normalization

In [ ]:
c19idxm_w = (c19idxm_w - c19idxm_w.min()) / (c19idxm_w.max() - c19idxm_w.min())
c19idxm_w

In [ ]:
c19idxm_w.plot(lw=3, figsize=(12, 4))
plt.show()

#### Group data by week or seven days

In [ ]:
grouper = pd.Grouper(axis=0, freq='7d')
c19idxm_w.groupby(grouper).mean().plot(lw=3, figsize=(12, 4))
plt.show()

In [ ]:
c19idxm_w.groupby('date').mean().rolling(7).mean().plot(lw=3, figsize=(12, 4))
plt.show()